In [1]:
source extend_bashkernel-2modes.source
set-global-evaluation-mode "my-script"
# (initialization cell, input hidden)

Ready.
Changing global mode to 'my-script'


In [ ]:
set-global-evaluation-mode "provided-script"

# Objective: Create and configure imagebuild job

## Task: Reset job

### Task Details

Same as previous

In [ ]:
# Task: imagebuild-reset-job
# Evaluation Mode: load
# (hidden load cell)
ssh -i ../mykeypair root@10.0.2.100

# Call to jenkins-cli client to delete the job.
java -jar jenkins-cli.jar -s http://localhost:8080/ delete-job tiny_web.imagebuild

### Example

Refer to the tiny_web.rspec job.

#### Provided script

In [ ]:
# Task: imagebuild-reset-job
# Evaluation Mode: provided-script

ssh -i ../mykeypair root@10.0.2.100

# Call to jenkins-cli client to delete the job.
java -jar jenkins-cli.jar -s http://localhost:8080/ delete-job tiny_web.imagebuild

### Check

In [ ]:
# Task: imagebuild-reset-job
# Evaluation Mode: check

## _Task: Create the job_

### Task details

Same as previous

In [ ]:
# Task: imagebuild-create-job
# Evaluation Mode: load
# (hidden load cell)
ssh -i ../mykeypair root@10.0.2.100

# Call to jenkins-cli client to create an empty job.
java -jar jenkins-cli.jar -s http://localhost:8080/ create-job tiny_web.imagebuild < ~/templates/config.xml

### Example

Refer to the tiny_web.rspec job.

#### Provided script

In [ ]:
# Task: imagebuild-create-job
# Evaluation Mode: provided-script

ssh -i ../mykeypair root@10.0.2.100

# Call to jenkins-cli client to create an empty job.
java -jar jenkins-cli.jar -s http://localhost:8080/ create-job tiny_web.imagebuild < ~/templates/config.xml

### Check

In [ ]:
# Task: imagebuild-create-job
# Evaluation Mode: check

## _Task: Configure git_

### Task details

Same as previous

In [ ]:
# Task: imagebuild-configure-git
# Evaluation Mode: load
# (hidden load cell)

### Example

Refer to the tiny_web.rspec job.

#### Provided script

In [ ]:
# (imagebuild-configure-git)
# Evaluation Mode: provided-script 

### Check

In [ ]:
# Task: imagebuild-configure-git
# Evaluation Mode: check

## _Task: Configure hipchat_

### Task details

Same as previous

In [ ]:
# Task: imagebuild-configure-hipchat
# Evaluation Mode: load
# (hidden load cell)

### Example

Refer to the tiny_web.rspec job.

#### Provided script

In [ ]:
# Task: imagebuild-configure-hipchat
# Evaluation Mode: provided-script

### Check

In [ ]:
# Task: imagebuild-configure-hipchat
# Evaluation Mode: check

## Task Configure parameters

### Task details

Jenkins allows for setting up parametrs which can later be used within the script. This can let us run the same job on enviroments where the task stays the same but certain variables can change depending on circumstances, results from previous job, who is building the job etc.

Add two string parameters and name them image_id and yum_host, insert appropriate values as the default value
where image_id is the name of the image that will be cloned and yum_host is the ip address of the machine that
provides the yum repository.

In [ ]:
# Task: imagebuild-configure-params
# Evaluation Mode: load
# (hidden load cell)

### Example

<a href="https://cloud.githubusercontent.com/assets/9693694/14378391/9d896656-fdaf-11e5-954a-1ee38858c0f0.png"><img width="20%" height ="20%" align="left" alt="imagebuild-configure-params-01" src="https://cloud.githubusercontent.com/assets/9693694/14378391/9d896656-fdaf-11e5-954a-1ee38858c0f0.png"></a>
<a href="https://cloud.githubusercontent.com/assets/9693694/14378392/9d8f4d32-fdaf-11e5-91ff-d41107ed001f.png"><img width="20%" height ="20%" align="left" alt="imagebuild-configure-params-02" src="https://cloud.githubusercontent.com/assets/9693694/14378392/9d8f4d32-fdaf-11e5-91ff-d41107ed001f.png"></a>

#### Provided script

In [ ]:
# Task: imagebuild-configure-params
# Evaluation Mode: provided-script

### Check

In [ ]:
# Task: imagebuild-configure-params
# Evaluation Mode: check

## _Task: Add build script_

### Task: Run a script as part of the build job

Same as previous

In [ ]:
# Task: imagebuild-configure-rbenv
# Evaluation Mode: load
# (hidden load cell)

### Example

Refer to the tiny_web.rspec job.

#### Provided script

In [ ]:
# Task: imagebuild-add-build-script
# Evaluation Mode: provided-script

### Check

In [ ]:
# Task: imagebuild-add-build-script
# Evaluation Mode: check

## _Task: Insert Template_

Execute the following cell to load in a template, where there are certain parts missing from the final script. The following tasks will work towards filling in the missing parts to complete the script.

#### Provided script

```
#!/bin/bash

set -eux
set -o pipefail

export PATH=$PATH:/opt/axsh/wakame-vdc/client/mussel/bin

SCRIPT_DIR="${WORKSPACE}/ciscripts/wakame-vdc"

# Task: Configure params

cd ${SCRIPT_DIR}

# wait for the instance to be running
. retry.sh

# vifs
network_id="nw-demo1"
security_group_id="sg-cicddemo"
vifs=vifs.json

# instance-specific parameter
cpu_cores="1"
hypervisor="openvz"
memory_size="512"
image_id="${IMAGE_ID}"
ssh_key_id="ssh-cicddemo"

# create an musselrc
cat <<EOS > ~/.musselrc
DCMGR_HOST=10.0.2.15
account_id=a-shpoolxx
EOS

# create an vifs
cat <<EOS > "${vifs}"
{
	"eth0":{"network":"${network_id}","security_groups":"${security_group_id}"}
}
EOS
## create database image

# db display name
display_name="db"

# create an instance
instance_id="$(
  mussel instance create \
  --cpu-cores    "${cpu_cores}"    \
  --hypervisor   "${hypervisor}"   \
  --image-id     "${image_id}"     \
  --memory-size  "${memory_size}"  \
  --ssh-key-id   "${ssh_key_id}"   \
  --vifs         "${vifs}"         \
  --display-name "${display_name}" \
  | egrep ^:id: | awk '{print $2}'
)"

: "${instance_id:?"should not be empty"}"
echo "${instance_id} is initializing..."

trap 'mussel instance destroy "${instance_id}"' ERR

# wait for the instance to be running
retry_until [[ '"$(mussel instance show "${instance_id}" | egrep -w "^:state: running")"' ]]
echo "${instance_id} is running"

# get instance ipaddr
ipaddr="$(
  mussel instance show "${instance_id}" \
  | egrep :address:  \
  | awk '{print $2}' \
  | tr '\n' ','
)"
: "${ipaddr:?"should not be empty"}"
ipaddr="${ipaddr%%,}"
echo "${instance_id} ipaddr: ${ipaddr}"

# wait for ssh to be ready
${SCRIPT_DIR}/instance-wait4ssh.sh  "${instance_id}"

# install package
${SCRIPT_DIR}/instance-exec.sh      "${instance_id}" < ${SCRIPT_DIR}/provision-imgdb.sh

# instance state: running -> halted
mussel instance poweroff --force false ${instance_id}
echo "${instance_id} is halting"

# wait for the instance to be halted
retry_until [[ '"$(mussel instance show "${instance_id}" | egrep -w "^:state: halted")"' ]]
echo "${instance_id} is halted"

# instance backup
DB_IMAGE_ID="$(mussel instance backup ${instance_id} --display-name db | egrep ^:image_id: | awk '{print $2}')"
echo "database image id: ${DB_IMAGE_ID}"

# wait for the image to be available
retry_until [[ '"$(mussel image show "${DB_IMAGE_ID}" | egrep -w "^:state: available")"' ]]
echo "${DB_IMAGE_ID} is available"

# instance destroy
mussel instance destroy "${instance_id}"
echo "${instance_id} is deleted"

## create app image

# app display name
display_name="app"

# create an instance
instance_id="$(
  mussel instance create \
   --cpu-cores    "${cpu_cores}"    \
   --hypervisor   "${hypervisor}"   \
   --image-id     "${image_id}"     \
   --memory-size  "${memory_size}"  \
   --ssh-key-id   "${ssh_key_id}"   \
   --vifs         "${vifs}"         \
   --display-name "${display_name}" \
  | egrep ^:id: | awk '{print $2}'
)"
: "${instance_id:?"should not be empty"}"
echo "${instance_id} is initializing..."

trap 'mussel instance destroy "${instance_id}"' ERR

# wait for the instance to be running
retry_until [[ '"$(mussel instance show "${instance_id}" | egrep -w "^:state: running")"' ]]
echo "${instance_id} is running"

# get instance ipaddr
ipaddr="$(
  mussel instance show "${instance_id}" \
  | egrep :address:  \
  | awk '{print $2}' \
  | tr '\n' ','
)"
: "${ipaddr:?"should not be empty"}"
ipaddr="${ipaddr%%,}"
echo "${instance_id} ipaddr: ${ipaddr}"

# wait for ssh to be ready
${SCRIPT_DIR}/instance-wait4ssh.sh  "${instance_id}"

# install package
${SCRIPT_DIR}/instance-exec.sh      "${instance_id}" \
		    YUM_HOST="${YUM_HOST}" \
		    bash -l < ${SCRIPT_DIR}/provision-imgapp.sh

# instance state: running -> halted
mussel instance poweroff --force false ${instance_id}
echo "${instance_id} is halting"

# wait for the instance to be halted
retry_until [[ '"$(mussel instance show "${instance_id}" | egrep -w "^:state: halted")"' ]]
echo "${instance_id} is halted"

# instance backup
APP_IMAGE_ID="$(mussel instance backup ${instance_id} --display-name app | egrep ^:image_id: | awk '{print $2}')"
echo "app image id: ${APP_IMAGE_ID}"

# wait for the image to be available
RETRY_WAIT_SEC=180 retry_until [[ '"$(mussel image show "${APP_IMAGE_ID}" | egrep -w "^:state: available")"' ]]
echo "${DB_IMAGE_ID} is available"

# instance destroy
mussel instance destroy "${instance_id}"
echo "${instance_id} is deleted"

# Task: Save params

```


## Task: Configure params 

### Task details

Set up the following variables
* `IMAGE_ID` to hold the image_id paramter we created earlier
* `YUM_HOST` to hold the yum_host paramter we created earlier.
* `WRITE_FILE` to represent a path that holds values generated by this job for future use.

In [ ]:
# Task: imagebuild-script-configure-params
# Evaluation Mode: load
# (hidden load cell)

### Examples

Creating variable:
```
variable_name=value
```

#### Provided script

In [ ]:
# Task: imagebuild-script-configure-params
# Evaluation Mode: provided-script

### Check

In [ ]:
# Task: imagebuild-script-configure-params
# Evaluation Mode: check

## Task: Save result


### Task details

The script will generate the following values
```
$DB_IMAGE_ID
$APP_IMAGE_ID
```
Reference to 204a on how to save values to a file, and save them in variable format.

In [ ]:
# Task: imagebuild-script-save-params
# Evaluation Mode: load
# (hidden load cell)

### Examples

To print output to files the "greater than" symbol can be used.
```
echo "content" > file_path
```
Refeence to 204a create image for more information

#### Provided script

In [ ]:
# Task: imagebuild-script-save-params
# Evaluation Mode: provided-script

### Check

In [ ]:
# Task: imagebuild-script-save-params
# Evaluation Mode: check